In [3]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
import pickle

In [4]:
train_dir = "data/train"
embedding_file = "face_embeddings.npy"
label_file = "face_labels.npy"

In [6]:
def build_cnn(input_shape=(112, 112, 3)):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(128, activation='relu')   # Embedding layer
    ])
    return model

cnn_model = build_cnn()
cnn_model.summary()

/home/shagor/Desktop/code/deep learning/env_dl/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1760605282.015665   51056 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 110, 110, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 55, 55, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 53, 53, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     4,718,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,844,992 (18.48 MB)

 Trainable params: 4,844,992 (18.48 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
def load_data(train_dir):
    X, y = [], []
    for person_name in os.listdir(train_dir):
        person_folder = os.path.join(train_dir, person_name)
        if not os.path.isdir(person_folder):
            continue
        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, (112, 112))
            img = img / 255.0
            X.append(img)
            y.append(person_name)
    return np.array(X), np.array(y)

print("Loading training data...")
X_train, y_train = load_data(train_dir)
print("Loaded:", X_train.shape, "images")

Loading training data...
Loaded: (15, 112, 112, 3) images


In [9]:
embeddings = cnn_model.predict(X_train)
print("Embeddings shape:", embeddings.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Embeddings shape: (15, 128)


In [10]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y_train)

In [12]:
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn.fit(embeddings, y_encoded)

,n_neighbors,3
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'euclidean'
,metric_params,None
,n_jobs,None


In [13]:
pickle.dump(knn, open("knn_model.pkl", "wb"))
pickle.dump(encoder, open("label_encoder.pkl", "wb"))
cnn_model.save("cnn_feature_extractor.h5")

print("Training complete & models saved successfully!")

Training complete & models saved successfully!


In [14]:
import cv2
import numpy as np
import os
import pickle
from tensorflow.keras.models import load_model

In [15]:
cnn_model = load_model("cnn_feature_extractor.h5")
knn = pickle.load(open("knn_model.pkl", "rb"))
encoder = pickle.load(open("label_encoder.pkl", "rb"))

test_dir = "data/test"

In [16]:

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (112, 112))
    img = img / 255.0
    return np.expand_dims(img, axis=0)

print("Testing on images...\n")

Testing on images...



In [17]:
for img_name in os.listdir(test_dir):
    img_path = os.path.join(test_dir, img_name)
    img = preprocess_image(img_path)
    
    # Extract features
    embedding = cnn_model.predict(img)[0]
    
    # Predict with KNN
    pred = knn.predict([embedding])[0]
    person_name = encoder.inverse_transform([pred])[0]
    
    print(f"Image: {img_name} → Predicted: {person_name}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Image: test_person4.jpeg → Predicted: person3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Image: p.jpg → Predicted: person2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Image: test_person3.png → Predicted: person3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Image: test_person2.jpeg → Predicted: person4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Image: test_person1.jpeg → Predicted: person3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Image: m.webp → Predicted: person1


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import pickle

cnn_model = load_model("cnn_feature_extractor.h5")
knn = pickle.load(open("knn_model.pkl", "rb"))
encoder = pickle.load(open("label_encoder.pkl", "rb"))

video = cv2.VideoCapture(0)

while True:
    ret, frame = video.read()
    if not ret:
        break

    face = cv2.resize(frame, (112, 112))
    img = face / 255.0
    img = np.expand_dims(img, axis=0)

    embedding = cnn_model.predict(img)[0]
    pred = knn.predict([embedding])[0]
    name = encoder.inverse_transform([pred])[0]

    cv2.putText(frame, name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()